# Shawn's Workspace

In [9]:
from modsim import *
from math import *
#takes velocity

In [10]:
mars = System(duration = 687, #days
distance_sun = 227.9 * 10 ^ 6, #km
distance_orbit = distance_sun * 3.14159
)

TypeError: unsupported operand type(s) for ^: 'float' and 'int'

In [ ]:
mars = System(x= 0 * m,
              y= 0 * m,
              radius = 3396200 * m,
              mass = .64171 * (10**24) * kg    
                )

rocket = System(x= -1 * mars.radius,
                y= 0 * m,
                dry_mass = 1433788 * kg,
                rho = 0.02 * kg/m**3, #needs function to update
                h = 0 * m, #height
                thrust = 22800000 * N,
                fuel_init = 1000000 * kg
               )


In [ ]:
vp = sqrt(2*G*sun.mass(saturn.radius/mars.radius/(mars.radius+saturn.radius)))
vp = 